In [22]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Import project zip file
import zipfile
zip_ref = zipfile.ZipFile("/content/drive/My Drive/ViolenceDetection.zip", 'r')

# extract in tmp folder
zip_ref.extractall("/tmp")
zip_ref.close()

In [0]:
# change current directory to tmp
import os
os.chdir("/tmp")

In [0]:
# import python files
import src
import settings
import Deploy
import Train
import Evaluate
%matplotlib inline

In [7]:
# install required packages
!pip install sk-video

     |████████████████████████████████| 2.3MB 34.6MB/s 


In [27]:
# reset all variables if needed
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


## Update Files

In [49]:
%%writefile src/net/G2D19_P2OF_ResHB_1LSTM.py

import tensorflow as tf
from src.net.NetBase import *
from src.layers.LayerHelper import *
from src.layers.BasicLayers import *
from src.layers.ResidualLayers import *
from src.layers.RNN import *
import settings.LayerSettings as layerSettings
import settings.DataSettings as dataSettings
import numpy as np

DARKNET19_MODEL_PATH = 'src//data//pretrainModels//darknet19//darknet19.pb'

class Net(NetworkBase):
	def __init__(self, inputImage_, batchSize_, unrolledSize_, isTraining_, trainingStep_):
		self._inputImage = inputImage_
		self._batchSize = batchSize_
		self._unrolledSize = unrolledSize_
		self._isTraining = isTraining_
		self._trainingStep = trainingStep_
	
		self._DROPOUT_PROB = 0.5
		self._NUMBER_OF_NEURONS_IN_LSTM = 1024

		self._dictOfInterestedActivations = {}

		if dataSettings.GROUPED_SIZE != 2:
			errorMessage = __name__ + " only take GROUPED_SIZE = 2;\n"
			errorMessage += "However, DataSettings.GROUPED_SIZE = " + str(dataSettings.GROUPED_SIZE)
			raise ValueError(errorMessage)

	def Build(self):
		darknet19_GraphDef = tf.GraphDef()

		'''
		      The CNN only take input shape [..., w, h, c].  Thus, move the UNROLLED_SIZE dimension
		    to merged with BATCH_SIZE, and form the shape: [b*u, w, h, c].
		'''
		convInput = tf.reshape(self._inputImage, [-1,
							  dataSettings.IMAGE_SIZE, dataSettings.IMAGE_SIZE, dataSettings.IMAGE_CHANNELS])

		with tf.name_scope("DarkNet19"):
			with open(DARKNET19_MODEL_PATH, 'rb') as modelFile:
				darknet19_GraphDef.ParseFromString(modelFile.read())
				listOfOperations = tf.import_graph_def(darknet19_GraphDef,
									input_map={"input": convInput},
#									return_elements=["2-maxpool", "32-leaky"])
									return_elements=["2-maxpool", "36-leaky"])
				pool2 = listOfOperations[0].outputs[0]
				lastOp = listOfOperations[-1]
				out = lastOp.outputs[0]
				out, updateOp0 = BatchNormalization('BN_0', out, isConvLayer_=True,
								     isTraining_=self._isTraining, currentStep_=self._trainingStep)
			

		opticalFlowOut, updateOF = self._buildOpticalFlowNet(pool2)
		print("opticalFlowOut.shape = ", opticalFlowOut.shape)  # shape = [b*u, 1024]

		with tf.name_scope("CNN"):
			'''
			    The input shape = [b, u, g, w, h, c]
			    after Conv, shape = [b*u*g, w', h', c']
			    here, decouple the Group dimension, shape = [b*u, g * w' * h' * c']
			'''
			print("In CNN:")
			print("\t darknetOutput.shape = ", out.shape)   # shape = [b*u*g, 7, 7, 1024]
			w, h, c = out.shape[1:]  # 7, 7, 1024
			out = tf.reshape( out,
					  [self._batchSize * self._unrolledSize, dataSettings.GROUPED_SIZE,
					  w, h, c])  # [b*u, g, 7, 7, 1024]
			out = tf.transpose(out, perm=[0, 2, 3, 4, 1])  # [b*u, 7, 7, 1024, g]
			print("\t after transpose, out.shape = ", out.shape)
			out = tf.reshape( out, 
					  [self._batchSize * self._unrolledSize,
					   w, h, c * dataSettings.GROUPED_SIZE])
			print("\t before ConcatConv, out.shape = ", out.shape)  # shape = [b*u, 7, 7, 1024*g]

			out, updateOp1 = ResidualHeadBlock('ResHeadBlock', out, [128, 64, 128], isTraining_=self._isTraining,
							trainingStep_=self._trainingStep, activationType_="LEAKY_RELU", isTrainable_=True)

			print("\t before Fc, out.shape = ", out.shape)
			out = tf.cond(self._isTraining, lambda: tf.nn.dropout(out, self._DROPOUT_PROB), lambda: out)

			out = FullyConnectedLayer('Fc1', out, numberOfOutputs_=1024)
			out, updateOp2 = BatchNormalization('BN_4', out, isConvLayer_=False,
							     isTraining_=self._isTraining, currentStep_=self._trainingStep)
			self._dictOfInterestedActivations['CNN'] = out
			print("\t Fc out.shape = ", out.shape)


		with tf.name_scope("Concat"):
			out = tf.concat([out, opticalFlowOut], axis=1)
			print("In Concat:")
			print("\t before Fc, out.shape = ", out.shape)
			out = FullyConnectedLayer('Fc', out, numberOfOutputs_=1024)
			out, updateOp3 = BatchNormalization('BN_5', out, isConvLayer_=False,
							     isTraining_=self._isTraining, currentStep_=self._trainingStep)
			'''
			    Note: For tf.nn.rnn_cell.dynamic_rnn(), the input shape of [1:] must be explicit.
			          i.e., one Can't Reshape the out by:
				  out = tf.reshape(out, [BATCH_SIZE, UNROLLED_SIZE, -1])
				  since '-1' is implicit dimension.
			'''
			featuresShapeInOneBatch = out.shape[1:].as_list()
			targetShape = [self._batchSize, self._unrolledSize] + featuresShapeInOneBatch
			out = tf.reshape(out, targetShape)
			self._dictOfInterestedActivations['Concat'] = out
			print("before LSTM, shape = ", out.shape)


		out, self._stateTensorOfLSTM_1, self._statePlaceHolderOfLSTM_1 = LSTM(	"LSTM_1",
											out,
											self._NUMBER_OF_NEURONS_IN_LSTM,
											isTraining_=self._isTraining,
											dropoutProb_=self._DROPOUT_PROB)

		self._dictOfInterestedActivations['LSTM'] = out

		with tf.name_scope("Fc_Final"):
			featuresShapeInOneBatch = out.shape[2:].as_list()
			targetShape = [self._batchSize * self._unrolledSize] + featuresShapeInOneBatch
			out = tf.reshape(out, targetShape)
			out = FullyConnectedLayer('Fc3', out, numberOfOutputs_=dataSettings.NUMBER_OF_CATEGORIES)
			self._logits = tf.reshape(out, [self._batchSize, self._unrolledSize, -1])

		self._updateOp = tf.group(updateOF, updateOp0, updateOp1, updateOp2, updateOp3)
		print()



	@property
	def logitsOp(self):
		return self._logits

	@property
	def updateOp(self):
		return self._updateOp


	def GetListOfStatesTensorInLSTMs(self):
		'''
		    You should Not Only sess.run() the net.logits, but also this listOfTensors
		    to get the States of LSTM.  And assign it to PlaceHolder next time.
		    ex:
			>> tupleOfResults = sess.run( [out] + net.GetListOfStatesTensorInLSTMs(), ...)
			>> listOfResults = list(tupleOfResults)
			>> output = listOfResults.pop(0)
			>> listOfStates = listOfResults

		    See GetFeedDictOfLSTM() method as well
		'''
		return [self._stateTensorOfLSTM_1]


	def GetFeedDictOfLSTM(self, BATCH_SIZE_, listOfPreviousStateValues_=None):
		'''
		      This function will return a dictionary that contained the PlaceHolder-Value map
		    of the LSTM states.
		      You can use this function as follows:
		    >> feed_dict = { netInput : batchOfImages }
		    >> feedDictOFLSTM = net.GetLSTM_Feed_Dict(BATCH_SIZE, listOfPreviousStateValues)
		    >> tupleOfOutputs = sess.run( [out] + net.GetListOfStatesTensorInLSTMs(),
						  feed_dict = feed_dict.update(feedDictOFLSTM) ) 
		    >> listOfOutputs = list(tupleOfOutputs)
		    >> output = listOfOutputs.pop(0)
		    >> listOfPreviousStateValues = listOfOutputs.pop(0)
		'''
		if listOfPreviousStateValues_ == None:
			'''
			    For the first time (or, the first of Unrolls), there's no previous state,
			    return zeros state.
			'''
			initialCellState = tuple( [np.zeros([BATCH_SIZE_, self._NUMBER_OF_NEURONS_IN_LSTM])] * 2 )
			initialCellState = tf.nn.rnn_cell.LSTMStateTuple(initialCellState[0], initialCellState[1])

			return {self._statePlaceHolderOfLSTM_1 : initialCellState }
		else:
			if len(listOfPreviousStateValues_) != 1:
				errorMessage = "len(listOfPreviousStateValues_) = " + str( len(listOfPreviousStateValues_) )
				errorMessage += "; However, the expected lenght is 1.\n"
				errorMessage += "\t Do you change the Network Structure, such as Add New LSTM?\n"
				errorMessage += "\t Or, do you add more tensor to session.run()?\n"

			return { self._statePlaceHolderOfLSTM_1 : listOfPreviousStateValues_[0] }


	def _buildOpticalFlowNet(self, inputTensor_):
		'''
		    The input shape = [b, u, g, w, h, c]
		    after Conv, shape = [b*u*g, w', h', c']
		    here, decouple the Group dimension, shape = [b*u, g * w' * h' * c']
		'''
		with tf.name_scope("OpticalFLow"):
			print("In OpticalFlow:")
			print("\t pool2.shape = ", inputTensor_.shape)  # shape = [b*u*g, 112, 112, 32]
			w, h, c = inputTensor_.shape[1:]  # 112, 112, 32
			out = tf.reshape( inputTensor_,
					  [self._batchSize * self._unrolledSize, dataSettings.GROUPED_SIZE,
					  w, h, c])  # [b*u, g, 112, 112, 32]
			out = tf.transpose(out, perm=[0, 2, 3, 4, 1])  # [b*u, 112, 112, 32, g]
			print("\t after transpose, out.shape = ", out.shape)
			out = tf.reshape( out, 
					  [self._batchSize * self._unrolledSize,
					   w, h, c * dataSettings.GROUPED_SIZE])
			print("\t before Conv2, out.shape = ", out.shape)  # shape = [b*u, 112, 112, 32*g]
			out = ConvLayer('Conv2', out, filterSize_=3, numberOfFilters_=64, stride_=1, padding_='SAME', isTrainable_=True)
			out, updateOp1 = BatchNormalization('BN2', out, isConvLayer_=True, isTraining_=self._isTraining,
								     currentStep_=self._trainingStep, isTrainable_=True)
			out = LeakyRELU('RELU2', out)
			out = MaxPoolLayer('Pool2', out, kernelSize_=2, stride_=2, padding_='SAME')

			out, updateOp2 = ResidualBlock('ResBlock', out, [128, 128, 64], isTraining_=self._isTraining,
							trainingStep_=self._trainingStep, activationType_="LEAKY_RELU", isTrainable_=True)

			out = MaxPoolLayer('Pool5', out, kernelSize_=2, stride_=2, padding_='SAME')

			print("\t before Fc, out.shape = ", out.shape)  # shape = [b*u, 28, 28, 128]
			out = tf.cond(self._isTraining, lambda: tf.nn.dropout(out, self._DROPOUT_PROB), lambda: out)

			out = FullyConnectedLayer('Fc_of', out, numberOfOutputs_=1024)
			out, updateOp3 = BatchNormalization('BN_of', out, isConvLayer_=False,
							     isTraining_=self._isTraining, currentStep_=self._trainingStep)
			self._dictOfInterestedActivations['OpticalFlow'] = out
			print("\t Fc final.shape = ", out.shape)
			updateOp = tf.group(updateOp1, updateOp2, updateOp3)
			return out, updateOp


Overwriting src/net/G2D19_P2OF_ResHB_1LSTM.py


In [0]:
%%writefile Train.py

#!/usr/bin/python3

from src.Evaluator import *
from src.Classifier import Classifier
from src.Trainer import Trainer
import settings.DataSettings as dataSettings
import time

class Main:
	def __init__(self):
		classifier = Classifier()
		classifier.Build()

		# Trainer, Evaluator
		print("Reading Training set...")
		self.trainer = Trainer(classifier)
		self.trainEvaluator = Evaluator("train", dataSettings.PATH_TO_TRAIN_SET_CATELOG, classifier)
		print("\t Done.\n")

		print("Reading Validation set...")
		self.validationEvaluator = Evaluator("validation", dataSettings.PATH_TO_VAL_SET_CATELOG, classifier)
		print("\t Done.\n")

		print("Reading Test set...")
		self.testEvaluator = Evaluator("test", dataSettings.PATH_TO_TEST_SET_CATELOG, classifier)
		print("\t Done.\n")

		# Summary
		summaryOp = tf.summary.merge_all()
		self.trainer.SetMergedSummaryOp(summaryOp)
		self.trainEvaluator.SetMergedSummaryOp(summaryOp)
		self.validationEvaluator.SetMergedSummaryOp(summaryOp)
		self.bestThreshold = None
		self.testEvaluator.SetMergedSummaryOp(summaryOp)

		# Time
		self._startTrainEpochTime = time.time()
		self._trainCountInOneEpoch = 0

		# Saver
		self.modelSaver = tf.train.Saver(max_to_keep=trainSettings.MAX_TRAINING_SAVE_MODEL)

		# Session
		self.session = tf.Session()
		init = tf.global_variables_initializer()
		self.session.run(init)

		self.trainer.SetGraph(self.session.graph)
		self.validationEvaluator.SetGraph(self.session.graph)

	def __del__(self):
		self.session.close()

	def Run(self):
		self.recoverFromPretrainModelIfRequired()

		self.calculateValidationBeforeTraining()
		self.resetTimeMeasureVariables()

		print("Path to save mode: ", trainSettings.PATH_TO_SAVE_MODEL)
		print("\nStart Training...\n")

		while self.trainer.currentEpoch < trainSettings.MAX_TRAINING_EPOCH:
			self.trainer.PrepareNewBatchData()
			self.trainer.Train(self.session)
			self._trainCountInOneEpoch += 1

			if self.trainer.isNewEpoch:
				print("Epoch:", self.trainer.currentEpoch, "======================================"
					+ "======================================"
					+ "======================================")

				self.printTimeMeasurement()
				self.trainer.PauseDataLoading()

				self.evaluateValidationSetAndPrint(self.trainer.currentEpoch)
				self.evaluateTrainingSetAndPrint(self.trainer.currentEpoch)

				if trainSettings.PERFORM_DATA_AUGMENTATION:
					# Preload TrainBatch while evaluate the TestSet
					self.trainer.ContinueDataLoading()

				self.evaluateTestSetAndPrint(self.trainer.currentEpoch)

				self.trainer.ContinueDataLoading()

				self.resetTimeMeasureVariables()

				if self.trainer.currentEpoch >= trainSettings.EPOCHS_TO_START_SAVE_MODEL:
					self.saveCheckpoint(self.trainer.currentEpoch)
		print("Optimization finished.")
		self.trainer.Release()
		self.trainEvaluator.Release()
		self.validationEvaluator.Release()
		self.testEvaluator.Release()



	def recoverFromPretrainModelIfRequired(self):
		if trainSettings.PRETRAIN_MODEL_PATH_NAME != "":
			print("Load Pretrain model from: " + trainSettings.PRETRAIN_MODEL_PATH_NAME)
			listOfAllVariables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
			variablesToBeRecovered = [ eachVariable for eachVariable in listOfAllVariables \
						   if eachVariable.name.split('/')[0] not in \
						   trainSettings.NAME_SCOPES_NOT_TO_RECOVER_FROM_CHECKPOINT ]
			modelLoader = tf.train.Saver(variablesToBeRecovered)
			modelLoader.restore(self.session, trainSettings.PRETRAIN_MODEL_PATH_NAME)

	def evaluateTrainingSetAndPrint(self, currentEpoch_):
		'''
		    Since the BATCH_SIZE may be small (= 4 in my case), its BatchLoss or BatchAccuracy
		    may be fluctuated.  Calculate the whole Training Loss instead.
		    Note: If one want to calculate the BatchLoss ONLY, use Trainer.EvaluateTrainLoss().
		'''
		startEvaluateTime = time.time()
		loss, frameAccuracy, threshold, videoAccuracy = self.trainEvaluator.Evaluate(	self.session,
												currentEpoch_=currentEpoch_,
												threshold_=self.bestThreshold)
		endEvaluateTime = time.time()

		self.printCalculationResults(jobType_='train', loss_=loss, frameAccuracy_=frameAccuracy,
					     isThresholdOptimized_=False,
					     threshold_=threshold, videoAccuracy_=videoAccuracy,
					     duration_=(endEvaluateTime-startEvaluateTime) )


	def calculateValidationBeforeTraining(self):
		if trainSettings.PRETRAIN_MODEL_PATH_NAME != "":
			print("Validation before Training ", "============================="
					+ "======================================"
					+ "======================================")
			self.evaluateValidationSetAndPrint(currentEpoch_=0)

	def evaluateValidationSetAndPrint(self, currentEpoch_):
		startEvaluateTime = time.time()
		loss, frameAccuracy, threshold, videoAccuracy = self.validationEvaluator.Evaluate(self.session,
												  currentEpoch_=currentEpoch_,
												  threshold_=None)
		endEvaluateTime = time.time()

		self.bestThreshold = threshold
		self.printCalculationResults(jobType_='validation', loss_=loss, frameAccuracy_=frameAccuracy,
					     isThresholdOptimized_=True,
					     threshold_=threshold, videoAccuracy_=videoAccuracy,
					     duration_=(endEvaluateTime-startEvaluateTime) )

	def evaluateTestSetAndPrint(self, currentEpoch_):
		startEvaluateTime = time.time()
		loss, frameAccuracy, threshold, videoAccuracy = self.testEvaluator.Evaluate(self.session,
											    currentEpoch_=currentEpoch_,
											    threshold_=self.bestThreshold)
		endEvaluateTime = time.time()

		self.printCalculationResults(jobType_='test', loss_=loss, frameAccuracy_=frameAccuracy,
					     isThresholdOptimized_=False,
					     threshold_=threshold, videoAccuracy_=videoAccuracy,
					     duration_=(endEvaluateTime-startEvaluateTime) )

	def printTimeMeasurement(self):
		timeForTrainOneEpoch = time.time() - self._startTrainEpochTime
		print("\t Back Propergation time measurement:")
		print("\t\t duration: ", "{0:.2f}".format(timeForTrainOneEpoch), "s/epoch")
		averagedTrainTime = timeForTrainOneEpoch / self._trainCountInOneEpoch
		print("\t\t average: ", "{0:.2f}".format(averagedTrainTime), "s/batch")
		print()

	def resetTimeMeasureVariables(self):
		self._startTrainEpochTime = time.time()
		self._trainCountInOneEpoch = 0

	def printCalculationResults(self, jobType_, loss_, frameAccuracy_, isThresholdOptimized_, threshold_, videoAccuracy_, duration_):
		floatPrecision = "{0:.4f}"
		print("\t "+jobType_+":")
		if isThresholdOptimized_:
			print("\t     loss:", floatPrecision.format(loss_),
				"     frame accuracy:", floatPrecision.format(frameAccuracy_),
				"     best frame threshold:", threshold_,
				"     video accuracy:", floatPrecision.format(videoAccuracy_),
				"     duration:", "{0:.2f}".format(duration_) + "(s)\n" )
		else:
			print("\t     loss:", floatPrecision.format(loss_),
				"     frame accuracy:", floatPrecision.format(frameAccuracy_),
				"     given frame threshold:", threshold_,
				"     video accuracy:", floatPrecision.format(videoAccuracy_),
				"     duration:", "{0:.2f}".format(duration_) + "(s)\n" )


	def saveCheckpoint(self, currentEpoch_):
		pathToSaveCheckpoint = os.path.join(trainSettings.PATH_TO_SAVE_MODEL, "save_epoch_" + str(currentEpoch_) )
		checkpointPathFileName = os.path.join(pathToSaveCheckpoint, "ViolenceNet.ckpt")
		self.modelSaver.save(self.session, checkpointPathFileName)


def main():
	main = Main()
	main.Run()


In [0]:
%%writefile Evaluate.py

#!/usr/bin/python3

import sys
import settings.EvaluationSettings as evalSettings
from src.Classifier import Classifier
from src.Evaluator import *
import time


def PrintResults(loss_, frameAccuracy_, isThresholdOptimized_, threshold_, videoAccuracy_, duration_):
    floatPrecision = "{0:.4f}"
    if isThresholdOptimized_:
        print("\t     loss:", floatPrecision.format(loss_),
              "     frame accuracy:", floatPrecision.format(frameAccuracy_),
              "     best frame threshold:", threshold_,
              "     video accuracy:", floatPrecision.format(videoAccuracy_),
              "     duration:", "{0:.2f}".format(duration_) + "(s)\n")
    else:
        print("\t     loss:", floatPrecision.format(loss_),
              "     frame accuracy:", floatPrecision.format(frameAccuracy_),
              "     given frame threshold:", threshold_,
              "     video accuracy:", floatPrecision.format(videoAccuracy_),
              "     duration:", "{0:.2f}".format(duration_) + "(s)\n")


def main(path_to_dataset_list = "datalists//evaluate.txt", find_threshold=False, threshold = 3):

    PATH_TO_DATA_SET_CATELOG = path_to_dataset_list
    classifier = Classifier()
    classifier.Build()
    evaluator = Evaluator("evaluate", PATH_TO_DATA_SET_CATELOG, classifier)

    with tf.Session() as session:
        init = tf.global_variables_initializer()
        session.run(init)

        print("Load Model from: ", evalSettings.PATH_TO_MODEL_CHECKPOINTS)
        modelLoader = tf.train.Saver()
        modelLoader.restore(session, evalSettings.PATH_TO_MODEL_CHECKPOINTS)

        startEvaluateTime = time.time()
        if find_threshold:
            print("Start evaluate: ", PATH_TO_DATA_SET_CATELOG, ", and find the best threshold...")
            loss, frameAccuracy, threshold, videoAccuracy = evaluator.Evaluate(session,
                                                                               currentEpoch_=0,
                                                                               threshold_=None)
            endEvaluateTime = time.time()
            PrintResults(loss_=loss, frameAccuracy_=frameAccuracy, isThresholdOptimized_=True,
                         threshold_=threshold, videoAccuracy_=videoAccuracy,
                         duration_=(endEvaluateTime - startEvaluateTime))

        else:
            threshold = threshold
            print("Start evaluate: ", PATH_TO_DATA_SET_CATELOG, ", with threshold : ", threshold)
            loss, frameAccuracy, threshold, videoAccuracy = evaluator.Evaluate(session,
                                                                               currentEpoch_=0,
                                                                               threshold_=threshold)
            endEvaluateTime = time.time()
            PrintResults(loss_=loss, frameAccuracy_=frameAccuracy, isThresholdOptimized_=False,
                         threshold_=threshold, videoAccuracy_=videoAccuracy,
                         duration_=(endEvaluateTime - startEvaluateTime))

    evaluator.Release()


In [0]:
%%writefile settings/DataSettings.py

import numpy as np

PATH_TO_TRAIN_SET_CATELOG = 'datalists//train.txt'
PATH_TO_VAL_SET_CATELOG = 'datalists//val.txt'
PATH_TO_TEST_SET_CATELOG = 'datalists//test.txt'

'''
    The input will be (BATCH_SIZE, UNROLLED_SIZE, GROUPED_SIZE, IMAGE_SIZE, IMAGE_SIZE, 3)
    'BATCH_SIZE': How many Videos should be used for a single step.
    'UNROLLED_SIZE':  How many Frames should be extracted from a video.
    'GROUPED_SIZE'  Some net need Many Frames (refered as the GROUPED_SIZE) to feed into the
		  net for one inference.  For example, the P2D19_1Fc_1LSTM take two frame
		  images as its input.
'''
GROUPED_SIZE = 2
IMAGE_SIZE = 224
IMAGE_CHANNELS = 3

'''
    In this project, categories = {NoFight, Fight}
'''
NUMBER_OF_CATEGORIES = 2


#########################
#  Uncommen Adjustment  #
#########################
NO_FIGHT_LABEL = [1., 0.]
FIGHT_LABEL = [0., 1.]


#####################
# Advenced Settings #
#####################
FLOAT_TYPE = np.float32
'''
    Following control the timeout of LoadData Thread.
    Recommand values:
	BATCH_SIZE=4, No DataAug, TIMEOUT_FOR_WAIT_QUEUE = 10
	BATCH_SIZE=4, DataAug, TIMEOUT_FOR_WAIT_QUEUE = 40
	BATCH_SIZE=20, No DataAug, TIMEOUT_FOR_WAIT_QUEUE = 20
	BATCH_SIZE=40, No DataAug, TIMEOUT_FOR_WAIT_QUEUE = 40
	BATCH_SIZE=40, DataAug, TIMEOUT_FOR_WAIT_QUEUE = 100
'''
TIMEOUT_FOR_WAIT_QUEUE = 100


In [0]:
%%writefile settings/EvaluationSettings.py

############################################################
#  Settings for both Evaluation in Train.py & Evaluate.py  #
############################################################
'''
    For evaluation, batchSize = 1 (i.e. pass the same video in the same batch).
'''
UNROLLED_SIZE = 50

#####################
# Advenced Settings #
#####################
WAITING_QUEUE_MAX_SIZE = 30
LOADED_QUEUE_MAX_SIZE = 15
NUMBER_OF_LOAD_DATA_THREADS=1


##############################
#  Settings for Evaluate.py  #
##############################

PATH_TO_MODEL_CHECKPOINTS = "datalists//save_epoch_12//ViolenceNet.ckpt"

In [0]:
%%writefile settings/TrainSettings.py

import tensorflow as tf
import settings.DataSettings as dataSettings

'''
      Following two variables control the shape of input
    data as the shape: [BATCH_SIZE*UNROLLED_SIZE, w, h, c].
    BATCH_SIZE: number of Videos in a batch.
    UNROLLED_SIZE: number of Frames in a Video.
      For the ConvNet part, the input will be the shape:
    [BATCH_SIZE*UNROLLED_SIZE, w, h, c].
      For the RNN part, the input will be the shape:
    [BATCH_SIZE, UNROLLED_SIZE, w, h, c] so that the
    tf.nn.rnn_cell.dynamic_rnn() can unroll the RNN.
      The output of the total network will be the shape:
    [BATCH_SIZE, UNROLLED_SIZE, NUMBER_OF_CATEGORIES]
'''
BATCH_SIZE = 4
UNROLLED_SIZE = 40
#BATCH_SIZE = 40
#UNROLLED_SIZE = 2

PRETRAIN_MODEL_PATH_NAME = "datalists//save_epoch_12//ViolenceNet.ckpt"
#PRETRAIN_MODEL_PATH_NAME = "temp/G2D19_P2OF_ResHB_1LSTM_dataAug_expLR/save_epoch_14/ViolenceNet.ckpt"

'''
    If one want to finetune, insert the LastLayer to the following list.
    ex: NAME_SCOPES_NOT_TO_RECOVER_FROM_CHECKPOINT = ['Conv4', 'Conv5']
'''
NAME_SCOPES_NOT_TO_RECOVER_FROM_CHECKPOINT = []

MAX_TRAINING_EPOCH = 2

EPOCHS_TO_START_SAVE_MODEL = 1
PATH_TO_SAVE_MODEL = "results//"
MAX_TRAINING_SAVE_MODEL = MAX_TRAINING_EPOCH
PERFORM_DATA_AUGMENTATION = True

def GetOptimizer(learningRate_):
	return tf.train.AdamOptimizer(learning_rate=learningRate_)

'''
    Following list three different LearningRate decay methods:
	1. _stairLearningRate(),
	2. _exponentialDecayLearningRate()
	3. _polynomialDecayLearningRate()
'''
def _stairLearningRate(currentEpoch_, currentStep_):
	#LIST_OF_EPOCH_LEARNING_RATE_PAIRS = [ (0, 1e-4), (5, 1e-5) ]
	LIST_OF_EPOCH_LEARNING_RATE_PAIRS = [ (0, 1e-6), (15, 5e-7), (25, 1e-7) ]
	#LIST_OF_EPOCH_LEARNING_RATE_PAIRS = [ (0, 3e-6), (5, 2.5e-6), (10, 2e-6), (15, 1.5e-6), (20, 1e-6) ]

	for eachPair in reversed(LIST_OF_EPOCH_LEARNING_RATE_PAIRS):
		if currentEpoch_ >= eachPair[0]:
			return eachPair[1]

	# If nothing matched, return the first pair.learningRate as default
	return LIST_OF_EPOCH_LEARNING_RATE_PAIRS[0][1] 


def _exponentialDecayLearningRate(currentEpoch_, currentStep_):
	'''
	    Exponential Decay:
		learningRate = INITIAL_LEARNING_RATE * DECAY_RATE ^ (currentStep_ / DECAY_STEP) + END_LEARNING_RATE
	'''
	INITIAL_LEARNING_RATE = 1e-5
	DECAY_RATE = 0.9
	NUMBER_OF_BATCHES_PER_EPOCH = 250
	NUMBER_OF_EPOCHS_PER_DECAY = 1
	DECAY_STEP = int(NUMBER_OF_BATCHES_PER_EPOCH * NUMBER_OF_EPOCHS_PER_DECAY)
	END_LEARNING_RATE = 0.0

	learningRate = INITIAL_LEARNING_RATE * DECAY_RATE ** (currentStep_ / DECAY_STEP) + END_LEARNING_RATE

	return learningRate

def _polynomialDecayLearningRate(currentEpoch_, currentStep_):
	'''
	    Polynomial Decay:
		step = min(currentStep_, MAX_STEPS)
		learningRate = (START_LEARNING_RATE - END_LEARNING_RATE) * (1 - step/MAX_STEPS)^(POWER) + END_LEARNING_RATE
	'''
	START_LEARNING_RATE = 2e-6
	END_LEARNING_RATE = 1e-7
	MAX_STEPS = MAX_TRAINING_EPOCH * 125
	POWER = 4



def GetLearningRate(currentEpoch_, currentStep_):
#	return _stairLearningRate(currentEpoch_, currentStep_)
	return _exponentialDecayLearningRate(currentEpoch_, currentStep_=currentStep_)



#####################
# Advenced Settings #
#####################
'''
    Following settings depend on (BATCH_SIZE, UNROLLED_SIZE, PERFORM_DATA_AUGMENTATION):
    if (4, 40, False), Recommend values:
	WAITING_QUEUE_MAX_SIZE = 60
	LOADED_QUEUE_MAX_SIZE = 30
	NUMBER_OF_LOAD_DATA_THREADS=2

    if (4, 40, True), Recommend values:
	WAITING_QUEUE_MAX_SIZE = 180
	LOADED_QUEUE_MAX_SIZE = 80
	NUMBER_OF_LOAD_DATA_THREADS=2

    if (40, 1, False), Recommend values:
	WAITING_QUEUE_MAX_SIZE = 180
	LOADED_QUEUE_MAX_SIZE = 80
	NUMBER_OF_LOAD_DATA_THREADS=4

    if (40, 1, True), Recommend values:
	WAITING_QUEUE_MAX_SIZE = 180
	LOADED_QUEUE_MAX_SIZE = 80
	NUMBER_OF_LOAD_DATA_THREADS=4

     Note: The "Averaged GetBatch Time" that printed while you train an epoch, should be
	   smaller than 0.001(s). Otherwise, increase NUMBER_OF_LOAD_DATA_THREADS.
'''
WAITING_QUEUE_MAX_SIZE = 180
LOADED_QUEUE_MAX_SIZE = 80
NUMBER_OF_LOAD_DATA_THREADS=4

MAX_GRADIENT_VALUE = 5.0
MIN_GRADIENT_VALUE = -5.0


In [0]:
# use to delete a file
# !rm src/net/G2D19_P2OF_ResHB_1LSTM.py

In [0]:
# use to view a file
%pycat Evaluate.py

In [56]:
Evaluate.main()


 Using Network:  src.net.G2D19_P2OF_ResHB_1LSTM 



FileNotFoundError: ignored

In [0]:
# list files in current working directory
!ls

drivefs_ipc.0  drivefs_ipc.0_shell


In [36]:
# zip project folder
!7z a tmp.7z tmp


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive:
  0M Scan           65 folders, 2289 files, 5752314554 bytes (5486 MiB)

Creating archive: tmp.7z

Items to compress: 2354

  0%      0% 31 + tmp/.idea/workspace.xml                                   0% 46 + tmp/datalists/darknet19.pb                                      1% 46 + tmp/datalists/darknet19.pb                                      1% 47 + tmp/datalists/evaluate.txt                                      1% 48 + tmp/datalists/save_epoch_12/ViolenceNet.ckpt.dat

In [0]:
# download project zip file
from google.colab import files
files.download("tmp.7z")

In [0]:
# delete a folder
# !rm -r tmp